## Directives

Nginx's **language** is made up of **directives**

Think of them as **instructions to the server to behave in one way or the other**

**Types of directives**

> 1) **Simple** : One statement ending with **semicolon**

> 2) **Block** (or **Context**): Number of simple directives grouped together in a block enclosed in curly braces ```{}```

> **Eg**: Take a look at **configuration file** specified under **--conf-path**, which is by default 
>> ```/etc/nginx/nginx.conf```

## nginx.conf

<div style="border:2px solid;margin:0 20px 0 20px"></div>

```bash
# user to operate nginx
user  nginx;

# should depend on number of cpu cores available on the machine
worker_processes  1;

# path to error log file
error_log  /var/log/nginx/error.log warn;

# path to pid file. 
# This file is created when server is up and destroyed when server is down
pid        /var/run/nginx.pid;

# finetune nginx performance
events {
    worker_connections  1024;
}


http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;

    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile        on;
    #tcp_nopush     on;

    keepalive_timeout  65;

    #gzip  on;

    # server settings
    include /etc/nginx/conf.d/*.conf;
}
```
<div style="border:2px solid;margin:0 20px 0 20px"></div>

**user nginx;**

> We need a user service account to run njinx

> Never use ROOT user

> User should not be able to open terminal and should not have home directory
```bash
#should have the following nginx user
vim /etc/passwd
nginx:x:101:101:nginx user,,,:/nonexistent:/bin/false
```


**worker_process 1;**

> Should depend on number of cpu cores available on the machine

**error_log  /var/log/nginx/error.log warn;**

> Path to error log file

> 'warn' is the level of verbosity by which errors will be logged.

> There are 6 levels of verbosity

>> info, notice, warn, error, crit, alert, emerg

**pid        /var/run/nginx.pid;**

> Records the **process id** of the service. 

> It is a file that contains a **single number**

> This file is created when server is up and destroyed when server is down

**events context**

> Context for **fine-tunning performance** of nginx

>Many directives can be placed in here. But only the important ones are specified here. Its better to leave the rest with default settings

> [Documentation](http://nginx.org/en/docs/ngx_core_module.html#example)

> a) **worker_connections**

>> specifies the **max number of concurrent connections** nginx (**master**) should handle.

>> Recommended value is **1024** for **single core machine**. If there are multiple cores, **multiply 1024 with num cores**

> b) **multi_accpet** (on|off)

>> Defines if **worker thread** can handle **more than one connection at a time**

>> This is **off** by default. This lets the **nginx master** decide the **worker process** to handle the request

> c) **Other directives**

>> use, accept_mutex, accept_mutex_delay

**http context**

> **NOTE**: This context is **not available if** the compile arguments have **--without-http**

> Provides the **configuration** file context in which the **HTTP server** directives are specified. 

> a) **include  /etc/nginx/mime.types;**

>> This **includes** another **directive file**. 

>> The **mime.types** contains the context '**types**', which specifies **how to treat files with different extensions** from webserver.
```bash
# if file name ends with html or htm or html, make the file type as text/html, which will be used by parser
types{
    ...
    text/html       html htm shtml; 
    ...
}
```
>> **USAGE**: 

>>> Suppose you have a file **config.json** in **/usr/share/nginx/html/**, when you hit **localhost/config.json**, the browser will know the type of file and render it as json

>>> Upon clicking **F12** and **network tab** --> **Response Headers**, you can find **content type** specified as **application/json** (corresponds to the mime type)

> b) **default_type** application/octet-stream

>> Specifies the type with which **unknown extensions should be handled**. 

>> **octet-stream** instructs the browser to **prompt for downloading the file** 

>c) **log_format main [FORMAT] **

>> The format of log. **main** is the name of log format.

>> In ```access_log  /var/log/nginx/access.log  main;```, we specify to use the log format **main**

>> **NOTE**: The log files in ```/var``` are usually deleted after the operation in completed. To view logs when server is running
```bash
tail -f /var/log/nginx/error.log
```

> d) **Important optimization arguments**:

>> **sendfile** : Optimizes interaction with **harddisk**.

>> **tcp_nopush**: Used with **sendfile**. Does not send packet unless the full packet size is reached. This is done to send all data in least number of packets

>> **tcp_nodelay**: Almost opposite of tcp_push. Traditionally, there would be a wait of **200ms** for the packet to be full before it is sent. But with this option **on** (default), data is **Sent immediately when its available**, thereby saving 200ms.

>> Using both **tcp_nopush** and **tcp_nodelay** can actually **optimize** data transfer. For eg, when sending a large file, the **last packet** may be half full and will be sent immediately, thereby **saving 200ms per file**

<img src="files/optimization.png" />

>> **[Reference1](https://docs.nginx.com/nginx/admin-guide/web-server/serving-static-content/)**

>> **[Reference2](https://thoughts.t37.net/nginx-optimization-understanding-sendfile-tcp-nodelay-and-tcp-nopush-c55cdd276765)**

> e) **keepalive_timeout**

>> The keepalive_timeout assigns the timeout for keep-alive connections with the client. Simply put, Nginx will close connections with the client after this period of time

>> This is important because, to load a webpage, the server makes a number of other request to load or download assets. If this parameter is not set, the browser should initiate new request to download assets

>> This will ensure **high performance when loading long pages**

> f) **gzip**

>> Compresses data that is being sent to browser.

>> The browser however needs to support gzip compression. Most modern browsers support them

>g) **include /etc/nginx/conf.d/*.conf;**

>> Include aditional settings

>> By default **/etc/nginx/conf.d/** folder contains one file ```default.conf``` containing the **Server context** (more on this in next section)

```python
        server {
            # listen on port 80
            listen       80;
            server_name  localhost;

            location / {
                # location of index file
                root   /usr/share/nginx/html;
                index  index.html index.htm;
            }
```

### Check for errors in config file

> ```nginx -t -c /etc/nginx/nginx.conf```